# **Obtención de máximos y mínmos gloabales por vector de característica: Normalización de ventanas de recorrido de extracción de características en tiempo real (en línea con baja latencia)**


------------------------------------------------------------
 Nombre del archivo  : step02_GlobalValuesForFeaturesWindowsNormalization_InferenceInRealTime.ipynb

 Autor               :   Jonathan Eduardo Castilla Zamora

 Github              :   https://github.com/JonathanCastilla/sEMG-RealTime-PatternRecognition-for-GripperControl

 Institución         :   Instituto Politécnico Nacional (IPN)

 Fecha de creación   :   Febrero/2025

 Última modificación :   Junio/2025

 Versión             :   1.3.1

 Descripción         :   El objetivo del presente algoritmo, como se precisó en todo el proceso de Reconocimiento de Patrones de las señales sEMG, es obtener los valores asociados a los máximos y mínimos globales para cada vector de característica para la normalización de tipo MinMax de las ventanas de recorrido de extracción de características en tiempo real (en línea con baja latencia) para el algoritmo de inferencia de comandos de muñeca de la actividad mioeléctrca de ambos canales analógicos (EDC y FDS). En particualr, como parte del proceso de clasificación de las muestras almacenadas asociados a los gestos de muñeca con el fin de traducir estos movimientos (WF, WE, HC y REP) en comandos para el control de movimiento de la pinza robótica en tiempo  real (en línea con baja latencia).
                        


------------------------------------------------------------

## Importación de bibliotecas

In [ ]:
import pandas as pd # Importa la biblioteca pandas, utilizada para la carga, manipulación y análisis de datos estructurados,
# especialmente en formato tabular mediante DataFrames.
import numpy as np # Importa numpy, una biblioteca fundamental para cálculos numéricos,
# especialmente útil para operar con arreglos y matrices.
from google.colab import files

## Definición de los archivos de datos correspondientes a cada gesto de muñeca para cada canal analógico:


* Extensor Común de los Dedos (EDC)

* Flexor Superficial de los Dedos (FDS)

### Protocolo de adquisición: M5sec

In [ ]:
# Definición de los archivos de datos correspondientes a cada gesto para ambos músculos (EDC y FDS)
# Archivos de datos M5sec
archivoWF_EDC_M5 = '/content/datosSensor_MAD_EDC_WF_M5sec_sEMG_nomMinMaxForSubject_FFC_LE_ESP32_IEMG-WL-MWL-Variance-DASDV_gestos_dataSet.csv' # Movimiento Wrist Flexion (WF) - Músculo EDC
archivoWF_FDS_M5 = '/content/datosSensor_MAD_FDS_WF_M5sec_sEMG_nomMinMaxForSubject_FFC_LE_ESP32_IEMG-WL-MWL-Variance-DASDV_gestos_dataSet.csv' # Movimiento Wrist Flexion (WF) - Músculo FDS
archivoWE_EDC_M5 = '/content/datosSensor_MAD_EDC_WE_M5sec_sEMG_nomMinMaxForSubject_FFC_LE_ESP32_IEMG-WL-MWL-Variance-DASDV_gestos_dataSet.csv' # Movimiento Wrist Extension (WE) - Músculo EDC
archivoWE_FDS_M5 = '/content/datosSensor_MAD_FDS_WE_M5sec_sEMG_nomMinMaxForSubject_FFC_LE_ESP32_IEMG-WL-MWL-Variance-DASDV_gestos_dataSet.csv' # Movimiento Wrist Extension (WE) - Músculo FDS
archivoHC_EDC_M5 = '/content/datosSensor_MAD_EDC_HC_M5sec_sEMG_nomMinMaxForSubject_FFC_LE_ESP32_IEMG-WL-MWL-Variance-DASDV_gestos_dataSet.csv' # Movimiento Hand Close (HC) - Músculo EDC
archivoHC_FDS_M5 = '/content/datosSensor_MAD_FDS_HC_M5sec_sEMG_nomMinMaxForSubject_FFC_LE_ESP32_IEMG-WL-MWL-Variance-DASDV_gestos_dataSet.csv'  # Movimiento Hand Close (HC) - Músculo FDS

### Protocolo de adquisición: M3sec

In [ ]:
# Archivos de datos M3sec
archivoWF_EDC_M3 = '/content/datosSensor_MAD_EDC_WF_M3sec_sEMG_nomMinMaxForSubject_FFC_LE_ESP32_IEMG-WL-MWL-Variance-DASDV_gestos_dataSet.csv' # Movimiento Wrist Flexion (WF) - Músculo EDC
archivoWF_FDS_M3 = '/content/datosSensor_MAD_FDS_WF_M3sec_sEMG_nomMinMaxForSubject_FFC_LE_ESP32_IEMG-WL-MWL-Variance-DASDV_gestos_dataSet.csv' # Movimiento Wrist Flexion (WF) - Músculo FDS
archivoWE_EDC_M3 = '/content/datosSensor_MAD_EDC_WE_M3sec_sEMG_nomMinMaxForSubject_FFC_LE_ESP32_IEMG-WL-MWL-Variance-DASDV_gestos_dataSet.csv' # Movimiento Wrist Extension (WE) - Músculo EDC
archivoWE_FDS_M3 = '/content/datosSensor_MAD_FDS_WE_M3sec_sEMG_nomMinMaxForSubject_FFC_LE_ESP32_IEMG-WL-MWL-Variance-DASDV_gestos_dataSet.csv' # Movimiento Wrist Extension (WE) - Músculo FDS
archivoHC_EDC_M3 = '/content/datosSensor_MAD_EDC_HC_M3sec_sEMG_nomMinMaxForSubject_FFC_LE_ESP32_IEMG-WL-MWL-Variance-DASDV_gestos_dataSet.csv' # Movimiento Hand Close (HC) - Músculo EDC
archivoHC_FDS_M3 = '/content/datosSensor_MAD_FDS_HC_M3sec_sEMG_nomMinMaxForSubject_FFC_LE_ESP32_IEMG-WL-MWL-Variance-DASDV_gestos_dataSet.csv'  # Movimiento Hand Close (HC) - Músculo FDS

### Archivos CSV unificados de características para ambos canales analógicos (EDC y FDS)

In [ ]:
# Archivos de reposo
archivoREP_EDC = '/content/reposo_EDC_unificado_dataSet.csv'
archivoREP_FDS = '/content/reposo_FDS_unificado_dataSet.csv'

## Normalización de vectores de características de las matrices resultantes del proceso de Extracción de Características mediante propuesta de recorridos de ventana y porcentajes de solapamiento

### Declaración de función para Normalización tipo MinMax

In [ ]:
# Función para normalizar cada columna de una matriz de manera independiente
def normalize_columns(matrix):
    # If the input is a DataFrame, convert it to a NumPy array
    if isinstance(matrix, pd.DataFrame):
        matrix = matrix.to_numpy()

    # Crear una copia de la matriz para no modificar la original
    normalized_matrix = np.zeros_like(matrix, dtype=float)

    # Recorrer cada columna y normalizarla
    for col in range(matrix.shape[1]):
        scaler = StandardScaler()
        normalized_matrix[:, col] = scaler.fit_transform(matrix[:, col].reshape(-1, 1)).flatten()

    return normalized_matrix

##  Carga y preparación de los conjuntos de características exportados en archivos CSV por tipo de gesto de muñeca (WF, WE, HC y REP), por músculo (canal analógico) (FDS y EDC) y por protocolo de adquisición (M3sec y M5sec).

1. Nuevamente, el algoritmo hereda la importación del conjunto de características relevantes resultantes del Análisis de Componentes Principales (PCA) mediante la definición de las rutas de acceso a los archivos CSV a cada uno de los documentos que contienen las características extraídas para cada gesto. De manera análoga, se cargan los datos correspondientes al gesto de reposo que presentan la peculiaridad de que el conjunto de datos se encuentra unificado por la naturaleza del gesto al que remite (REP: reposo).

  En particular, se retoma la consideración de la caracterización de los gestos de muñeca que aluden a tres acciones motoras voluntarias (Flexión de muñeca, Extensión de muñeca y Mano cerrada, abreviadas como WF, WE, y HC, respectivamente), así como una condición de reposo (REP), dónde estas señales fueron registradas simultáneamente desde dos músculos de la zona del antebrazo: Extensor Común de los dedos (EDC) y Flexor Superficial de los dedos (FDS), de tal modo que cada gesto fue almacenado en archivos separados según el músculo y la duración del segmento de adquisición: 3 segundos (M3sec) y 5 segundos (M5sec) asociado al periodo de tiempo realización sostenido (prolongado) del gesto sostenido. Estos archivos son cargados como objetos DataFrame de pandas y concatenados verticalmente para formar un único conjunto unificado por gesto y músculo.

2. Tras la carga de datos de las características (variables) representativas resultantes del algoritmo del Análisis de Componentes Principales (PCA), se elimina nuevamente la primera columna, que corresponde al índice o número de muestra (número de ventana de recorrido) conservando únicamente las columnas con las características extraídas del Análisis de Componentes Principales (PCA) dado que, como el caso de los algoritmos explicados con anterioridad, no aporta información de interés para la implementación del clasificador de Aprendizaje de tipo supervisado ni del subdominio del Aprendizaje Profundo (Deep Learning): Perceptrón Multicapa (MLP). De igual forma, el conjunto de datos se convierten en matrices NumPy para facilitar su manipulación posterior.

3. Las matrices resultantes son convertidas a estructuras de tipo numpy.ndarray para facilitar operaciones matemáticas subsecuentes. Luego, todas las matrices son recortadas de manera uniforme hasta el mínimo número de filas (valores de las ventanas de características extraídas) entre todas ellas, garantizando que cada clase tenga igual número de muestras y evitar sesgos de entrenamiento, como lo implementado en algoritmos anteriores. Este enfoque permite asegurar que cada clase contenga el mismo número de ejemplos. Tras finalizar esto, se concatenan verticalmente las muestras para formar dos matrices completas: una para el músculo Extensor Digitorum Communis (EDC) y otra para el Flexor Digitorum Superficialis (FDS).

In [ ]:
# Cargar archivos en DataFrames
WF_EDC = pd.concat([
    pd.read_csv(archivoWF_EDC_M5),
    pd.read_csv(archivoWF_EDC_M3)
], ignore_index=True)

WF_FDS = pd.concat([
    pd.read_csv(archivoWF_FDS_M5),
    pd.read_csv(archivoWF_FDS_M3)
], ignore_index=True)

WE_EDC = pd.concat([
    pd.read_csv(archivoWE_EDC_M5),
    pd.read_csv(archivoWE_EDC_M3)
], ignore_index=True)

WE_FDS = pd.concat([
    pd.read_csv(archivoWE_FDS_M5),
    pd.read_csv(archivoWE_FDS_M3)
], ignore_index=True)

HC_EDC = pd.concat([
    pd.read_csv(archivoHC_EDC_M5),
    pd.read_csv(archivoHC_EDC_M3)
], ignore_index=True)

HC_FDS = pd.concat([
    pd.read_csv(archivoHC_FDS_M5),
    pd.read_csv(archivoHC_FDS_M3)
], ignore_index=True)

# Reposo se mantiene igual
REP_EDC = pd.read_csv(archivoREP_EDC)
REP_FDS = pd.read_csv(archivoREP_FDS)

# Extraer características
caracteristicas = WF_EDC.columns[1:].tolist()
print("Características:", caracteristicas)

# Eliminar primera columna (número de muestra)
WF_EDC = WF_EDC.iloc[:, 1:]
WF_FDS = WF_FDS.iloc[:, 1:]
WE_EDC = WE_EDC.iloc[:, 1:]
WE_FDS = WE_FDS.iloc[:, 1:]
HC_EDC = HC_EDC.iloc[:, 1:]
HC_FDS = HC_FDS.iloc[:, 1:]
REP_EDC = REP_EDC.iloc[:, 1:]
REP_FDS = REP_FDS.iloc[:, 1:]

# Convertir a matrices NumPy
matrices_EDC = [WF_EDC.to_numpy(), WE_EDC.to_numpy(), HC_EDC.to_numpy(), REP_EDC.to_numpy()]
matrices_FDS = [WF_FDS.to_numpy(), WE_FDS.to_numpy(), HC_FDS.to_numpy(), REP_FDS.to_numpy()]

# Determinar mínimo de filas
min_filas = min(mat.shape[0] for mat in matrices_EDC + matrices_FDS)

# Recortar todas al mínimo
matrices_EDC = [mat[:min_filas, :] for mat in matrices_EDC]
matrices_FDS = [mat[:min_filas, :] for mat in matrices_FDS]

# Concatenar verticalmente
P_EDC = np.vstack(matrices_EDC)
P_FDS = np.vstack(matrices_FDS)

print("Forma final P_EDC:", P_EDC.shape)
print("Forma final P_FDS:", P_FDS.shape)

Características: ['IEMG', 'WL', 'MWL', 'Variance', 'DASDV']
Forma final P_EDC: (26360, 5)
Forma final P_FDS: (26360, 5)


### Declaración de función para obtención de los valores máximos y mínimos de cada vector de características para normalización en tiempo real de las ventanas de recorrido de extracción de características tipo MinMax

In [ ]:
def get_min_max(data):
    """
    Obtiene los valores mínimos y máximos de cada columna (característica) en el conjunto de datos.

    Parámetros:
        data (numpy.ndarray): Matriz de datos donde cada fila es un segmento y cada columna es una característica.

    Retorna:
        min_values (numpy.ndarray): Array con los valores mínimos de cada columna.
        max_values (numpy.ndarray): Array con los valores máximos de cada columna.
    """
    min_values = np.min(data, axis=0)  # Mínimo de cada columna
    max_values = np.max(data, axis=0)  # Máximo de cada columna
    return min_values, max_values

## Preparación de los archivos CSV obtenidos de la Matriz de Correlación..

  * Se concatenan las matrices de carcterísticas asociadas a cada músculo (canal analógico) en una única matriz que representa todo el conjunto de
 características a analizar. El propósito de esta concatenación es obtener los valores asociados a los máximos y mínimos globales de cada vector de característica sin importar su canal analógico para la normalización en tiempo real de las ventanas de recorrido de extracción de características en el algoritmo de inferencia en tiempo real (en línea con baja latencia). Esta concatenación permite obtener dichos valores para las características independientemente del canal analógico del que provengan (FDS o EDC), por lo que este enfoque permite realizar un análisis generalizado de las características relevantes del conjunto de datos.


In [ ]:
# Concatenación de características de ambos músculos en una sola matriz de datos
data = np.vstack((P_EDC, P_FDS))

print(f"\nForma de los conjuntos de datos totales:")
print(f"P_EDC: {P_EDC.shape}")
print(f"P_FDS: {P_FDS.shape}")
print(f"data: {data.shape}")
# print(data)


Forma de los conjuntos de datos totales:
P_EDC: (26360, 5)
P_FDS: (26360, 5)
data: (52720, 5)


In [ ]:
print(np.isnan(data).sum())
data = np.nan_to_num(data, nan=0.0)
P_EDC = np.nan_to_num(P_EDC, nan=0.0)
P_FDS = np.nan_to_num(P_FDS, nan=0.0)

0


## Obtención de máximos y mínimos globales de los vectores de características

In [ ]:
min_values, max_values = get_min_max(data)
globalMinMaxValuesNormalization = np.array([min_values, max_values])
print("Valores mínimos de cada característica:", globalMinMaxValuesNormalization[0])
print("Valores máximos de cada característica:", globalMinMaxValuesNormalization[1])

Valores mínimos de cada característica: [8.87864823e-01 0.00000000e+00 0.00000000e+00 1.20705916e-35
 0.00000000e+00]
Valores máximos de cada característica: [3.11316901e+02 2.49690211e+00 2.92565056e+00 1.34176081e-01
 5.16329287e-02]


In [ ]:
min_values_EDC, max_values_EDC = get_min_max(P_EDC)
print("Valores mínimos de cada característica:", min_values_EDC)
print("Valores máximos de cada característica:", max_values_EDC)

Valores mínimos de cada característica: [8.87864823e-01 0.00000000e+00 0.00000000e+00 1.20705916e-35
 0.00000000e+00]
Valores máximos de cada característica: [3.11316901e+02 2.49690211e+00 2.92565056e+00 1.34176081e-01
 5.16329287e-02]


In [ ]:
minMaxValuesNormalization_EDC = np.array([min_values_EDC, max_values_EDC])
print(minMaxValuesNormalization_EDC[0])

[8.87864823e-01 0.00000000e+00 0.00000000e+00 1.20705916e-35
 0.00000000e+00]


In [ ]:
min_values_FDS, max_values_FDS = get_min_max(P_FDS)

print("Valores mínimos de cada característica:", min_values_FDS)
print("Valores máximos de cada característica:", max_values_FDS)

Valores mínimos de cada característica: [2.21198157e+00 0.00000000e+00 0.00000000e+00 2.71588310e-35
 0.00000000e+00]
Valores máximos de cada característica: [1.74462926e+02 1.26452906e+00 1.18125960e+00 4.37338457e-02
 2.41925381e-02]


In [ ]:
minMaxValuesNormalization_FDS = np.array([min_values_FDS, max_values_FDS])
print(minMaxValuesNormalization_FDS[0])

[2.21198157e+00 0.00000000e+00 0.00000000e+00 2.71588310e-35
 0.00000000e+00]


## Guardado y descarga de los archivos tipo NumPy de los máximos y mínimos globales de cada vector de característica sin considerar su canal analógico (EDC y FDS)

In [ ]:
# Guardar el array en un archivo .npy
np.save('minMaxValuesNormalization_EDC_sEMGnomMinMax_FFC_LE_ESP32_featuresSet_PCA+MatCorr_reduced1.npy', minMaxValuesNormalization_EDC)
np.save('minMaxValuesNormalization_FDS_sEMGnomMinMax_FFC_LE_ESP32_featuresSet_PCA+MatCorr_reduced1.npy', minMaxValuesNormalization_FDS)
# Descargar el archivo a tu PC
files.download('minMaxValuesNormalization_EDC_sEMGnomMinMax_FFC_LE_ESP32_featuresSet_PCA+MatCorr_reduced1.npy')
files.download('minMaxValuesNormalization_FDS_sEMGnomMinMax_FFC_LE_ESP32_featuresSet_PCA+MatCorr_reduced1.npy')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Guardar el array en un archivo .npy
np.save('globalMinMaxValuesNormalization_sEMGnomMinMax_FFC_LE_ESP32_featuresSet_PCA+MatCorr_reduced1.npy', globalMinMaxValuesNormalization)

# Descargar el archivo a tu PC
files.download('globalMinMaxValuesNormalization_sEMGnomMinMax_FFC_LE_ESP32_featuresSet_PCA+MatCorr_reduced1.npy')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>